## WIDER Dataset - DataLoader

In [1]:
# !pip install --upgrade torch --quiet

In [1]:
import json
import io
import boto3
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import numpy as np
from PIL import Image
from torchvision.transforms import transforms, RandAugment
import torch
import tarfile
import os
import getpass
import s3fs
import json
from urllib.parse import urlparse
from data.data_loader import load_cifar10, load_cifar100, load_imagenet, load_prof, load_wider


### Data Download - Do not need to repeat this step

In [ ]:
# Option 1
# # !aws s3 cp s3://210bucket/wider_attribute_image.tgz ./wider_attribute_image.tgz
# Option 2
# s3 = boto3.client('s3')
# s3.download_file('210bucket', 'wider_attribute_image.tgz', 'wider_attribute_image.tgz')

In [ ]:
# # Extract the .tgz file
# with tarfile.open('wider_attribute_image.tgz', 'r:gz') as tar:
#     tar.extractall(path="local_wider")

In [6]:
# # Upload the image files to s3 Bukcet
# s3 = boto3.client('s3')

# def upload_dir_to_s3(bucket_name, s3_folder, local_dir):
#     for root, dirs, files in os.walk(local_dir):
#         for file in files:
#             local_path = os.path.join(root, file)
#             relative_path = os.path.relpath(local_path, local_dir)
#             s3_path = os.path.join(s3_folder, relative_path)
            
#             s3.upload_file(local_path, bucket_name, s3_path)

# upload_dir_to_s3('210bucket', 'WIDER/', 'WIDER/Image/')


### DataLoader

In [7]:
# ######################## now available using "from data.data_loader import" #########################

# class DataSet(Dataset):
#     def __init__(self, ann_files, augs, img_size, dataset, undersampe=False):
#         self.dataset = dataset
#         self.ann_files = ann_files
#         self.augment = self.augs_function(augs, img_size)
#         self.transform = transforms.Compose(
#             [
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=[0, 0, 0], 
#                                      std=[1, 1, 1])
#             ]
#         )
#         self.anns = []
#         self.s3_client = boto3.client('s3') 
#         self.load_anns()
#         print(self.augment)

#         if self.dataset == "wider":
#             self.transform = transforms.Compose(
#                 [
#                     transforms.ToTensor(),
#                     transforms.Normalize(mean=[0.5, 0.5, 0.5], 
#                                          std=[0.5, 0.5, 0.5])
#                 ]
#             )

#     def extract_label_from_filename(self, file_name):
#         # Split the path and extract the part with '--'
#         parts = file_name.split('/')
#         for part in parts:
#             if '--' in part:
#                 # Extract the numeric part before '--'
#                 label = part.split('--')[0]
#                 if label.isdigit():
#                     return int(label)  # Return as an integer
#         raise ValueError(f"Label not found in file name: {file_name}")

        

#     def augs_function(self, augs, img_size):            
#         t = []
#         if 'randomflip' in augs:
#             t.append(transforms.RandomHorizontalFlip())
#         if 'ColorJitter' in augs:
#             t.append(transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0))
#         if 'resizedcrop' in augs:
#             t.append(transforms.RandomResizedCrop(img_size, scale=(0.7, 1.0)))
#         if 'RandAugment' in augs: # need to review RandAugment()
#             t.append(RandAugment())
#             # t.append(transforms.RandomApply([
#             #     transforms.RandomRotation(degrees=10),
#             #     transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
#             #     transforms.RandomPerspective(distortion_scale=0.05)
#             # ], p=0.5))

#         t.append(transforms.Resize((img_size, img_size)))
    
#         return transforms.Compose(t)

#     def load_anns(self):
#         s3_client = boto3.client('s3')
#         self.anns = []
#         for ann_file in self.ann_files:
#             bucket, key = self.parse_s3_path(ann_file)
#             response = s3_client.get_object(Bucket=bucket, Key=key)
#             json_data = json.loads(response['Body'].read())
#             for image in json_data['images']:
#                 file_name = image['file_name']
#                 label = self.extract_label_from_filename(file_name)  # Use the new method
#                 for target in image['targets']:
#                     ann = {
#                         'img_path': f's3://210bucket/WIDER/{file_name}',
#                         'bbox': target['bbox'],
#                         'label': label,  
#                         'target': target['attribute']
#                     }
#                     self.anns.append(ann)
#         print(f"Loaded annotations: {len(self.anns)}")


#     def undersample_anns(self):
#         # Shuffle annotations before undersampling
#         random.shuffle(self.anns)

#         # Count the instances per class
#         class_counts = {}
#         for ann in self.anns:
#             label = self.extract_label(ann['img_path'])  # Assuming this method returns the class label
#             class_counts[label] = class_counts.get(label, 0) + 1

#         # Find the minimum class count
#         min_count = min(class_counts.values())

#         # Perform undersampling
#         undersampled_anns = []
#         current_counts = {label: 0 for label in class_counts}
#         for ann in self.anns:
#             label = self.extract_label(ann['img_path'])
#             if current_counts[label] < min_count:
#                 undersampled_anns.append(ann)
#                 current_counts[label] += 1

#         # Update the annotations to the undersampled list
#         self.anns = undersampled_anns
        
#     def __len__(self):
#         return len(self.anns)

#     def __getitem__(self, idx):
#         idx = idx % len(self)
#         ann = self.anns[idx]
    
#         if not isinstance(ann, dict) or "img_path" not in ann:
#             raise ValueError(f"Annotation at index {idx} is not a dictionary with an 'img_path' key: {ann}")
    
#         bucket, key = self.parse_s3_path(ann["img_path"])
#         try:
#             response = self.s3_client.get_object(Bucket=bucket, Key=key)
#             img = Image.open(io.BytesIO(response['Body'].read())).convert("RGB")
#         except self.s3_client.exceptions.NoSuchKey:
#             print(f"File not found: s3://{bucket}/{key}")
#             img = self.get_placeholder_image()
#             return None 
    
#         x, y, w, h = ann['bbox']
#         img = img.crop((x, y, x + w, y + h))
#         img = self.augment(img)
#         img = self.transform(img)
    
#         label = torch.tensor(ann['label'], dtype=torch.long)    
#         target = torch.tensor(ann['target'], dtype=torch.float32)
    
#         message = {
#             "label": label,  
#             "target": target,  
#             "img": img 
#         }
    
#         return message


#     def get_placeholder_image(self):
#         return Image.new('RGB', (256, 256), color = 'gray')

#     @staticmethod
#     def parse_s3_path(s3_path):
#         if not s3_path.startswith("s3://"):
#             raise ValueError(f"Invalid S3 path: {s3_path}")
#         s3_path = s3_path[5:]
#         bucket, key = s3_path.split('/', 1)
#         return bucket, key

In [8]:
# ######################## now available using "from data.data_loader import" #########################

# def load_wider(batch_size=64, subset_size=None):

#     train_augs = ['randomflip', 'ColorJitter', 'resizedcrop', 'RandAugment']
#     test_augs = []  
#     img_size = 256 
    
#     train_dataset = DataSet(
#         ann_files=['s3://210bucket/wider_attribute_annotation/wider_attribute_trainval.json'],  
#         augs=train_augs,
#         img_size=img_size,
#         dataset='wider'
#     )
    
#     test_dataset = DataSet(
#         ann_files=['s3://210bucket/wider_attribute_annotation/wider_attribute_test.json'], 
#         augs=test_augs,
#         img_size=img_size,
#         dataset='wider'
#     )
    
#     if subset_size is not None:
#         train_indices = np.random.choice(len(train_dataset), subset_size, replace=False)
#         test_indices = np.random.choice(len(test_dataset), subset_size, replace=False)
        
#         train_subset = Subset(train_dataset, train_indices)
#         test_subset = Subset(test_dataset, test_indices)
        
#         train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
#         test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)
#     else:
#         train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#         test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#     return train_loader, test_loader


In [21]:
######################## now available using "from data.data_loader import" #########################

class DataSet(Dataset):
    def __init__(self, ann_files, augs, img_size, dataset, undersampe=False):

        self.label_mapping = {old_label: new_label for new_label, old_label in enumerate(sorted(class_labels))}
        
        self.dataset = dataset
        self.ann_files = ann_files
        self.augment = self.augs_function(augs, img_size)
        self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(mean=[0, 0, 0], 
                                     std=[1, 1, 1])
            ]
        )
        self.anns = []
        self.s3_client = boto3.client('s3') 
        self.load_anns()
        print(self.augment)

        if self.dataset == "wider":
            self.transform = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], 
                                         std=[0.5, 0.5, 0.5])
                ]
            )

    def extract_label_from_filename(self, file_name):
        # Split the path and extract the part with '--'
        parts = file_name.split('/')
        for part in parts:
            if '--' in part:
                # Extract the numeric part before '--'
                label = part.split('--')[0]
                if label.isdigit():
                    return int(label)  # Return as an integer
        raise ValueError(f"Label not found in file name: {file_name}")

        if label is not None:
            remapped_label = self.label_mapping[label]
            return remapped_label
        else:
            raise ValueError(f"Label could not be extracted from path: {img_path}")


    def augs_function(self, augs, img_size):            
        t = []
        if 'randomflip' in augs:
            t.append(transforms.RandomHorizontalFlip())
        if 'ColorJitter' in augs:
            t.append(transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0))
        if 'resizedcrop' in augs:
            t.append(transforms.RandomResizedCrop(img_size, scale=(0.7, 1.0)))
        if 'RandAugment' in augs: # need to review RandAugment()
            t.append(RandAugment())
            # t.append(transforms.RandomApply([
            #     transforms.RandomRotation(degrees=10),
            #     transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
            #     transforms.RandomPerspective(distortion_scale=0.05)
            # ], p=0.5))

        t.append(transforms.Resize((img_size, img_size)))
    
        return transforms.Compose(t)

    def load_anns(self):
        s3_client = boto3.client('s3')
        self.anns = []

        for ann_file in self.ann_files:
            bucket, key = self.parse_s3_path(ann_file)
            response = s3_client.get_object(Bucket=bucket, Key=key)
            json_data = json.loads(response['Body'].read())

        
        for image in json_data['images']:
            file_name = image['file_name']
            label = self.extract_label_from_filename(file_name)  # Use the new method
            for target in image['targets']:
                ann = {
                    'img_path': f's3://210bucket/WIDER/{file_name}',
                    'bbox': target['bbox'],
                    'label': label,  
                    'targets': target['attribute']
                }
                self.anns.append(ann)
                
        print(f"Loaded annotations: {len(self.anns)}")

        # for ann_file in self.ann_files:
        #     bucket, key = self.parse_s3_path(ann_file)
        #     response = s3_client.get_object(Bucket=bucket, Key=key)
        #     json_data = json.loads(response['Body'].read())
        #     self.anns += json_data
            
        #     display(json_data)
            
        # print(f"Loaded annotations: {len(self.anns)}")

    
    def undersample_anns(self):
        # Shuffle annotations before undersampling
        random.shuffle(self.anns)

        # Count the instances per class
        class_counts = {}
        for ann in self.anns:
            label = self.extract_label_from_filename(ann['img_path'])  # Assuming this method returns the class label
            class_counts[label] = class_counts.get(label, 0) + 1

        # Find the minimum class count
        min_count = min(class_counts.values())

        # Perform undersampling
        undersampled_anns = []
        current_counts = {label: 0 for label in class_counts}
        for ann in self.anns:
            label = self.extract_label_from_filename(ann['img_path'])
            if current_counts[label] < min_count:
                undersampled_anns.append(ann)
                current_counts[label] += 1

        # Update the annotations to the undersampled list
        self.anns = undersampled_anns
        
    def __len__(self):
        return len(self.anns)

    def __getitem__(self, idx):
        idx = idx % len(self)
        ann = self.anns[idx]
    
        if not isinstance(ann, dict) or "img_path" not in ann:
            raise ValueError(f"Annotation at index {idx} is not a dictionary with an 'img_path' key: {ann}")
    
        bucket, key = self.parse_s3_path(ann["img_path"])
        try:
            response = self.s3_client.get_object(Bucket=bucket, Key=key)
            img = Image.open(io.BytesIO(response['Body'].read())).convert("RGB")
        except self.s3_client.exceptions.NoSuchKey:
            print(f"File not found: s3://{bucket}/{key}")
            img = self.get_placeholder_image()
            return None 
    
        x, y, w, h = ann['bbox']
        img = img.crop((x, y, x + w, y + h))
        img = self.augment(img)
        img = self.transform(img)
        img_path = io.BytesIO(response['Body'].read())
        label = self.extract_label_from_filename(ann["img_path"]) 

        # attributes_list = [target['attribute'] for target in ann['targets']]
        # summed_attributes = [max(sum(attribute), 0) for attribute in zip(*attributes_list)]

        # display(ann['targets'])
        
        # if isinstance(ann['targets'], list):
        #     attributes_list = [target['attribute'] for target in ann['targets']]
        #     summed_attributes = [max(sum(attribute), 0) for attribute in zip(*attributes_list)]

        # elif isinstance(ann['targets'], int):
        #     # If 'target' is a single integer, handle it accordingly
        #     summed_attributes = [ann['targets']]
        # else:
        #     # Handle other cases or unknown structures
        #     print(f"Unknown structure for 'target' in ann: {ann}")
        #     summed_attributes = []

        # target_tensor = torch.tensor(summed_attributes, dtype=torch.float32)

        # target = torch.Tensor(summed_attributes)
        target = torch.tensor(ann['targets'], dtype=torch.float32)
        
        message = {
            "label": label,  
            "target": target,  
            "img": img 
        }
    
        return message


    def get_placeholder_image(self):
        return Image.new('RGB', (256, 256), color = 'gray')

    @staticmethod
    def parse_s3_path(s3_path):
        if not s3_path.startswith("s3://"):
            raise ValueError(f"Invalid S3 path: {s3_path}")
        s3_path = s3_path[5:]
        bucket, key = s3_path.split('/', 1)
        return bucket, key


    def extract_label(self, img_path):
        original_label = None
    
        if "WIDER/Image/train" in img_path:
            label_str = img_path.split("WIDER/Image/train/")[1].split("/")[0]
            original_label = int(label_str.split("--")[0])
        elif "WIDER/Image/test" in img_path:
            label_str = img_path.split("WIDER/Image/test/")[1].split("/")[0]
            original_label = int(label_str.split("--")[0])
        elif "WIDER/Image/val" in img_path:  # Handle validation images
            label_str = img_path.split("WIDER/Image/val/")[1].split("/")[0]
            original_label = int(label_str.split("--")[0])
    
        if original_label is not None:
            remapped_label = self.label_mapping[original_label]
            return remapped_label
        else:
            raise ValueError(f"Label could not be extracted from path: {img_path}")


In [22]:
def custom_collate(batch):
    # Filter out any None items in the batch
    batch = [item for item in batch if item is not None]
    # If after filtering the batch is empty, handle this case by either returning an empty tensor or raising an exception
    if len(batch) == 0:
        # Option 1: Return a placeholder tensor (adapt the shape to match your data)
        # return torch.tensor([]), torch.tensor([])
        # Option 2: Raise an exception
        raise ValueError("Batch is empty after filtering out None items.")
    return torch.utils.data.dataloader.default_collate(batch)


In [23]:
# Hyperparameters
num_classes = 30
batch_size = 372
num_workers = 4

class_labels = [0, 1, 3, 4, 6, 7, 11, 15, 17, 18, 19, 20, 22, 25, 27, 28, 30, 31, 33, 35, 36, 37, 39, 43, 44, 50, 51, 54, 57, 58]
class_labels_new = torch.tensor([i for i in range(len(class_labels))])



In [24]:
train_file = ['s3://210bucket/wider_attribute_annotation/wider_attribute_test.json']
test_file = ['s3://210bucket/wider_attribute_annotation/wider_attribute_trainval.json']

train_dataset = DataSet(train_file, augs=['RandAugment'], img_size=226, dataset='wider')
test_dataset = DataSet(test_file, augs=[], img_size=226, dataset='wider')


# trainloader = DataLoader(train_dataset, 
#                           batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=custom_collate)
# testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=custom_collate)


Loaded annotations: 29179
Compose(
    RandAugment(num_ops=2, magnitude=9, num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    Resize(size=(226, 226), interpolation=bilinear, max_size=None, antialias=warn)
)
Loaded annotations: 28345
Compose(
    Resize(size=(226, 226), interpolation=bilinear, max_size=None, antialias=warn)
)


In [25]:
# trainloader, testloader  = load_wider(batch_size=64)

In [26]:
train_dataset[0]

{'label': 0,
 'target': tensor([ 1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1., -1.]),
 'img': tensor([[[ 0.3020,  0.3020,  0.3176,  ..., -0.4588, -0.4275, -0.4275],
          [ 0.9686,  0.9686,  0.9608,  ..., -0.4902, -0.4902, -0.4902],
          [ 0.9686,  0.9686,  0.9686,  ..., -0.0431, -0.0118, -0.0118],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],
 
         [[ 0.3098,  0.3098,  0.3255,  ..., -0.4510, -0.4196, -0.4196],
          [ 0.9765,  0.9765,  0.9686,  ..., -0.4824, -0.4824, -0.4824],
          [ 0.9686,  0.9686,  0.9686,  ..., -0.0353, -0.0039, -0.0039],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1

In [124]:
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=custom_collate)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=custom_collate)


## New WIDER DataLoader

In [1]:
import json
import io
import boto3
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import numpy as np
from PIL import Image
import torchvision
from torchvision.transforms import transforms, RandAugment
import torch
import tarfile
import os
import getpass
import s3fs
import json
from urllib.parse import urlparse
from data.data_loader import load_cifar10, load_cifar100, load_imagenet, load_prof, load_wider


In [2]:
# Hyperparameters
learning_rate = 0.001379 # 0.096779
num_epochs = 15 # 200
temperature = 4.0
alpha = 0.9
momentum = 0.9
num_classes = 30
step_size = 30
gamma = 0.1
beta = 0.0
patience = 7  # for early stopping

batch_size = 372
num_workers = 4

# class_labels = [0, 1, 3, 4, 6, 7, 11, 15, 17, 18, 19, 20, 22, 25, 27, 28, 30, 31, 33, 35, 36, 37, 39, 43, 44, 50, 51, 54, 57, 58]
# class_labels_new = torch.tensor([i for i in range(len(class_labels))])

In [3]:
torchvision.transforms.RandAugment()

RandAugment(num_ops=2, magnitude=9, num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)

In [4]:
trainloader, testloader = load_wider(batch_size, num_workers) 

Compose(
    RandAugment(num_ops=2, magnitude=9, num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    Resize(size=(226, 226), interpolation=bilinear, max_size=None, antialias=warn)
)
Compose(
    Resize(size=(226, 226), interpolation=bilinear, max_size=None, antialias=warn)
)
